<a href="https://colab.research.google.com/github/chinmay-sh/StockAnalysisAI/blob/main/TickerAnalysisAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gradio -q
!pip install yfinance -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import yfinance as yf
import gradio as gr
import google.generativeai as genai
from google.colab import userdata
import requests
import math
import json

In [3]:
ALPHAVANTAGE_API_KEY = userdata.get('ALPHAVANTAGE_API_KEY')

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')


In [4]:
# prompt: generate a function for calling alphavantage api, sample api call for news sentiment: https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=demo, also limit to top 10 latest articles and only care about sentiment for the provided ticker

def get_news_sentiment(ticker):
    """
    Retrieves news sentiment for a given ticker from the Alpha Vantage API.

    Args:
        ticker: The stock ticker symbol (e.g., "AAPL").
        apikey: Your Alpha Vantage API key.

    Returns:
        A list of dictionaries, where each dictionary represents a news article
        and contains the article's title, summary, and sentiment score for the
        specified ticker.  Returns an empty list if there's an error or no data.
    """

    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={ALPHAVANTAGE_API_KEY}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        data = response.json()

        if "Information" in data or data["Information"]:
            print(f"Error from Alpha Vantage API: {data['Information']}")
            return []

        if "feed" not in data or not data["feed"]:
            print("No news sentiment data found for this ticker.")
            return []

        # Limit to the top 10 latest articles and filter by ticker sentiment
        articles = []
        for item in data["feed"][:10]:
            for sentiment in item["ticker_sentiment"]:
                if sentiment["ticker"] == ticker:
                    articles.append({
                        "title": item["title"],
                        "summary": item["summary"],
                        "url": item["url"],
                        "sentiment_score": sentiment["ticker_sentiment_score"],
                        "sentiment_label": sentiment["ticker_sentiment_label"]
                    })
                    break  # Stop once we find the sentiment for the ticker

        return articles

    except requests.exceptions.RequestException as e:
        print(f"Error fetching news sentiment data: {e}")
        return []
    except KeyError as e:
        print(f"Error parsing API response: Missing key {e}")
        return []


In [5]:
# prompt: python text to dict

def output_json_parser(text):
    try:
        # Attempt to parse the JSON string directly
        text = text.split("```json")[1].split("```")[0]
        # print(text)
        return json.loads(text)
    except json.JSONDecodeError:
        try:
            # If direct parsing fails, try removing potential prefixes/suffixes
            # and then parsing. This assumes the JSON is embedded within the text.
            # Adjust the slicing indices if your prefix/suffix differs.
            start_index = text.find('{')
            end_index = text.rfind('}') + 1

            if start_index != -1 and end_index != 0 and start_index < end_index :
              text = text[start_index:end_index]
              return json.loads(text)
            else:
              return {} # Or raise an exception, depending on how you want to handle it.
        except json.JSONDecodeError:
            print("Invalid JSON format")
            return {} # Or raise an exception if you prefer.

In [6]:
def predict_stock_price(ticker):
    ticker = ticker.upper()
    # Get historical data
    stock_data = yf.Ticker(ticker)
    hist = stock_data.history(period="5d")


    hist_closing_prices = {}
    for key, value in hist["Close"].to_dict().items():
        hist_closing_prices[str(key).split(" ")[0]] = math.floor(value)

    # Get current price
    current_price = stock_data.info['currentPrice']

    # Get news sentiment from Alpha Vantage
    news_sentiment_data_alphavantage = get_news_sentiment(ticker)

    # Prepare prompt for LLM
    system_instructions = """
    You are a stock/cryptocurrency price and news sentiment analysis model.

    Input will be stock/cryptocurrency ticker along with current price, 5 days of historical daily price, news sentiment average from AlphaVantage API for latest 10 news articles, and 10 latest news articles will also be input.

    Your task is to output a summary of those 10 news articles along with estimates of closing price for next 3 days. This estimate should be based on price trends along with news sentiment analysis and knowledge gained from news articles.

    Inputs:
    ===============
    Ticker: [string]
    current_price: [float]
    historical_price_data: [list of historical closing prices for last 5 days week]
    news_sentiment_average_alphavantage: [Average of news sentiment scores for latest 10 news articles from AlphaVantage API (x, where x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish)]
    news_articles_list: [list of data of Latest 10 news articles related to ticker from AlphaVantage API]


    Outputs (in JSON format):
    ===============
    predicted_prices: [list of predicted closing prices for next 3 days]
    summary_of_news_articles: [list of summary of each of latest 10 news articles in 100 words, also escaping any special character which might result in errors while parsing]
    collective_summary: [summary of all 10 news articles in 250 words]
    explanation: [explanation of how the model arrived at the predicted prices and collective summary, in 500 words, also escaping any special character which might result in errors while parsing]

    Example for prompts:
    ===============
    Input:
    ------
    ticker: "AAPL"
    current_price: 150.0
    historical_price_data: {'2024-10-28': 426,'2024-10-29': 431,'2024-10-30': 432,'2024-10-31': 406,'2024-11-01': 410}
    news_sentiment_average_alphavantage: 0.2
    news_articles_list: [{'title': 'Why Qorvo Stock Tanked This Week',
        'summary': 'The chip supplier really had a fiscal second quarter to forget.',
        'url': 'https://www.fool.com/investing/2024/11/01/why-qorvo-stock-tanked-this-week/',
        'sentiment_score': '-0.036857',
        'sentiment_label': 'Neutral'},
      {'title': 'Fitness App Market to Grow by USD 55.86 Billion  ( 2024-2028 )  as Health Management Needs Rise; AI-Redefined Market Landscape Report - Technavio',
        'summary': 'NEW YORK, Nov. 1, 2024 /PRNewswire/ -- Report with market evolution powered by AI - The global fitness app market size is estimated to grow by USD 55.86 billion from 2024-2028, according to Technavio. The market is estimated to grow at a CAGR of 17.78% during the forecast period.',
        'url': 'https://www.benzinga.com/pressreleases/24/11/n41697134/fitness-app-market-to-grow-by-usd-55-86-billion-2024-2028-as-health-management-needs-rise-ai-redef',
        'sentiment_score': '0.11726',
        'sentiment_label': 'Neutral'},
      {'title': '"Rule Breaker Investing" Mailbag: Nonpartisan Pre-Election Special',
        'summary': 'We answer your questions.',
        'url': 'https://www.fool.com/investing/2024/11/01/rule-breaker-investing-mailbag-nonpartisan-pre-ele/',
        'sentiment_score': '0.102002',
        'sentiment_label': 'Neutral'},
      {'title': 'Breaking Down Magnificent 7 Earnings Results',
        'summary': "We've received quarterly results from six members of the Magnificent 7, with AI-favorite NVIDIA now the lone standing member. How did they perform?",
        'url': 'https://www.zacks.com/commentary/2362899/breaking-down-magnificent-7-earnings-results',
        'sentiment_score': '0.115134',
        'sentiment_label': 'Neutral'},
      {'title': 'PayPal Looks Beyond Payments',
        'summary': "We also talk about some lesser-known names trying to follow Berkshire Hathaway's path.",
        'url': 'https://www.fool.com/investing/2024/11/01/paypal-looks-beyond-payments/',
        'sentiment_score': '0.101414',
        'sentiment_label': 'Neutral'},
      {'title': 'Three Stocks: Trump Media & Technology, Intel, and Globalstar',
        'summary': "DJT Trump Media & Technology ( DJT ) stock took a sharp dive today, falling over 10%. With a trading volume of 61.4 million shares, nearly three times its 90-day average of 21.3 million, the stock's volatility shows no sign of cooling down.",
        'url': 'https://moneymorning.com/2024/11/01/three-stocks-trump-media-technology-intel-and-globalstar/',
        'sentiment_score': '0.008491',
        'sentiment_label': 'Neutral'},
      {'title': 'Crude Oil Moves Higher; Apple Shares Slide After Q4 Results - Proto Labs  ( NYSE:PRLB ) ',
        'summary': 'U.S. stocks traded higher toward the end of trading, with the Nasdaq Composite gaining more than 100 points on Friday. The Dow traded up 0.67% to 42,043.38 while the NASDAQ surged 0.75% to 18,230.53. The S&P 500 also rose, gaining, 0.47% to 5,732.21. Consumer discretionary shares rose by 2.3% on ...',
        'url': 'https://www.benzinga.com/news/earnings/24/11/41693153/crude-oil-moves-higher-apple-shares-slide-after-q4-results',
        'sentiment_score': '-0.050722',
        'sentiment_label': 'Neutral'},
      {'title': "If You'd Invested $1,000 in Apple Stock 10 Years Ago, Here's How Much You'd Have Today",
        'summary': 'Apple has delivered incredible returns for long-term investors.',
        'url': 'https://www.fool.com/investing/2024/11/01/if-youd-invested-1000-in-apple-stock-10-years-ago/',
        'sentiment_score': '0.483551',
        'sentiment_label': 'Bullish'},
      {'title': "Friday's Top 5 Trending Stocks: What's The Scoop On Amazon, Trump Media, Intel? - Intel  ( NASDAQ:INTC ) , Super Micro Computer  ( NASDAQ:SMCI ) , Amazon.com  ( NASDAQ:AMZN ) , Trump Media & Technology  ( NASDAQ:DJT ) ",
        'summary': "Amazon.com, Inc. AMZN, Trump Media & Technology Group Corp. DJT, Intel Corp. INTC, Apple, Inc. and Super Micro Computer, Inc. SMCI are the top five trending tickers on Stocktwits Friday. Here's a look at what grabbed retail investors' attention.",
        'url': 'https://www.benzinga.com/news/earnings/24/11/41691123/fridays-top-5-trending-stocks-whats-the-scoop-on-amazon-trump-media-intel',
        'sentiment_score': '0.372595',
        'sentiment_label': 'Bullish'},
      {'title': "Apple Q4 Results 'Better-Than-Feared': Analysts Highlight iPhone 16 Demand, Apple Intelligence Opportunity After 'Rock Solid Quarter' - Apple  ( NASDAQ:AAPL ) ",
        'summary': "Apple Inc AAPL beat analysts' fourth-quarter revenue and earnings estimates, but guidance that came in below estimates has analysts questioning the future outlook after Thursday's report. Bank of America analyst Wamsi Mohan reiterated a Buy rating on Apple with a $256 price target.",
        'url': 'https://www.benzinga.com/analyst-ratings/analyst-color/24/11/41690576/apple-q4-results-better-than-feared-analysts-highlight-iphone-16-demand-apple-intel',
        'sentiment_score': '0.34258',
        'sentiment_label': 'Somewhat-Bullish'}]

    Output:
    ------
    future_prices_3_days: {'2024-11-02': price, '2024-11-03': price, '2024-11-04': price}
    summary_of_news_articles: ['summary 1 ...', 'summary 2 ...', ...,  'summary 10 ...']
    collective_summary: "summary of all 10 news articles in 250 words..."
    explanation_of_prediction: "explanation of how the model arrived at the future prices and the collective summary..."
    """
    model = 'gemini-1.5-flash' # @param {type: "string"} ["gemini-1.5-pro", "gemini-1.5-flash"]
    temperature = 0.5 # @param {type: "slider", min: 0, max: 2, step: 0.05}

    genai.configure(api_key=GEMINI_API_KEY)

    model = genai.GenerativeModel(model, system_instruction=system_instructions)
    config = genai.GenerationConfig(temperature=temperature)

    prompt = f"""
    Input:
    ------
    ticker: {ticker}
    current_price: {stock_data.info['currentPrice']}
    historical_price_data: {hist_closing_prices}
    news_sentiment_average_alphavantage: 0.2
    news_articles_list: {news_sentiment_data_alphavantage}

    Outputs (in JSON format):
    ===============
    predicted_prices: [list of predicted closing prices for next 3 days]
    summary_of_news_articles: [list of summary of each of latest 10 news articles]
    collective_summary: [summary of all 10 news articles in 250 words]
    explanation_of_prediction: [explanation of how the model arrived at the future prices and the collective summary]
    """

    response = model.generate_content(contents=[prompt], generation_config=config)

    output_json = output_json_parser(response.text)

    return output_json, ticker, stock_data.info['currentPrice'], hist_closing_prices, news_sentiment_data_alphavantage



In [8]:
with gr.Blocks() as app:
    gr.Markdown("# Stock Ticker Analysis & Prediction App")

    ticker_input = gr.Textbox(
        label="Enter Ticker for analysis (support only for NASDAQ tickers / Yahoo Finance)",
        placeholder="e.g., aapl/msft",
    )

    analyze_button = gr.Button("Analyze", variant='primary')


    gr.Markdown("## Stock Details")
    ticker_output = gr.Textbox(label="Ticker")
    current_price_output = gr.Textbox(label="Current Price")
    historical_price_data_output = gr.JSON(label="Historical Price Data")
    news_sentiment_alphavantage_output = gr.JSON(label="News Sentiment AlphaVantage")

    prediction_output = gr.JSON(label="LLM Prediction")

    analyze_button.click(
        predict_stock_price,
        inputs=[ticker_input],
        outputs=[prediction_output, ticker_output, current_price_output, historical_price_data_output, news_sentiment_alphavantage_output],
    )

# Launch the app
app.launch(share=True, debug=True, show_error=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a4c806ee55c89c57fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Error from Alpha Vantage API: Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a4c806ee55c89c57fc.gradio.live


In [29]:
output_json, ticker, stdata, hist_closing_prices, news_sentiment_data_alphavantage = predict_stock_price("msft")

print(output_json)

Error from Alpha Vantage API: Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.


KeyboardInterrupt: 